## Below are the requirements of Dataset Generation

Day of Billing Date: 100 days

Brand: Aiba, Biba, Ciba

Article: 10 digits, may not be unique

Site: ST01, ST02 upto ST40

Site Area : A, B, C

MRP: (999, 9999)

Promotion Name: P1, P2, P3...P40

Region: R1, R2, R3, R4

POS Billing Quantity: (-1 :2%, 0:8%, 1:90%)

POS Cost: POS Billin Quantity * rand(45% to 50%)

POS Gross Sales: POS Billing Quantity x MRP

POS Total Discount: POS Gross Sales * rand (0% to 30%)

POS Tax amount: (POS GRoss Sales - POS Total Discount)*5%

POS Net Sales WOT: POS Gross Sales - POS Total Discount - POS Tax amount

POS RGM : POS Net Sales WOT - POS Cost

Vendor : V1, V2, V3...V10

Vendor Code: CV1, CV2...corresponding to vendor V1, V2...

Fiber: Cotton, Silk, Cotton/Silk, Linen, Modal 

Fabric:Plian: 80%, Georgette: 2%, ChiffoN: 2% ,Organza: 2%, Crepe: 4%, Satin: 7%, Other:3%

Brand Tag: Concat of Brand + Fabric + Saree

Wash Care Instructions: Silk, Cotton Silk: Dry Clean only; cotton, Linen, Modal: Handwash Separately

C Technique: C1....C10

Broad_craft: B1: for C1 to C3, B2: C4 to C6, B3: C7 to C9, B4: C10

Handloom Mark: yes,no

Silk Mark: For fiber of Silk : yes, other: no

Type: Woven, Printed, Tie-Dyed, Embroidered

Style Code: Corresponding to Article: 7 charaters

craft Code: 3 Codes for every C Technique, eg. for C1 the craft codes are C11, C12, C13

Site Name: Concat of Site and Site Area

Bin: [1000,2000,3000,4000,5000,6000,7000,8000,9000,10000]


In [5]:
import pandas as pd
import random

# Set the number of rows for the dataset
num_rows = 1000

# Initialize lists to store data for each field
billing_date = [pd.to_datetime("2023-01-01") + pd.Timedelta(days=random.randint(0, 100)) for _ in range(num_rows)]
brand = random.choices(["Aiba", "Biba", "Ciba"], k=num_rows)
article = [str(random.randint(1000000000, 9999999999)) for _ in range(num_rows)]
site = [f"ST{random.randint(1, 40):02d}" for _ in range(num_rows)]
site_area = random.choices(["A", "B", "C"], k=num_rows)
mrp = [random.randint(999, 9999) for _ in range(num_rows)]
promotion_name = random.choices([f"P{i}" for i in range(1, 41)], k=num_rows)  # Corrected parentheses position
#region = random.choices(["R1", "R2", "R3", "R4"], k=num_rows)
# Create a mapping between sites and regions
site_to_region_mapping = {site_code: random.choice(["R1", "R2", "R3", "R4"]) for site_code in set(site)}
# Assign regions based on the mapping
region = [site_to_region_mapping[site_code] for site_code in site]
pos_billing_quantity = random.choices([-1, 0, 1], weights=[0.02, 0.08, 0.9], k=num_rows)
pos_cost = [qty * random.uniform(0.45, 0.5) for qty in pos_billing_quantity]
pos_gross_sales = [qty * price for qty, price in zip(pos_billing_quantity, mrp)]
pos_total_discount = [sales * random.uniform(0, 0.3) for sales in pos_gross_sales]
pos_tax_amount = [(sales - discount) * 0.05 for sales, discount in zip(pos_gross_sales, pos_total_discount)]
pos_net_sales_wot = [sales - discount - tax for sales, discount, tax in zip(pos_gross_sales, pos_total_discount, pos_tax_amount)]
pos_rgm = [net_sales - cost for net_sales, cost in zip(pos_net_sales_wot, pos_cost)]
vendor = [f"V{i}" for i in range(1, 11)] * ((num_rows + 9) // 10)  # Repeating vendors to match the number of rows
fiber = random.choices(["Cotton", "Silk", "Cotton/Silk", "Linen", "Modal"], weights=[0.8, 0.02, 0.02, 0.02, 0.04], k=num_rows)
fabric = random.choices(["Plain", "Georgette", "Chiffon", "Organza", "Crepe", "Satin", "Other"],
                        weights=[0.8, 0.02, 0.02, 0.02, 0.04, 0.07, 0.03], k=num_rows)
type = random.choices(["Woven", "Printed", "Tie-Dyed", "Embroidered"], k=num_rows)
style_code = [a[:7] for a in article]
craft_code = random.choices(["C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9", "C10"], k=num_rows)
site_name = [f"{s}{a}" for s, a in zip(site, site_area)]
bin_edges = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
bin_labels = [f"{bin_edges[i]}-{bin_edges[i+1]-1}" for i in range(len(bin_edges)-1)]
bin = pd.cut(mrp, bins=bin_edges, labels=bin_labels, right=False)

# Create the DataFrame
data = {
    "Billing Date": billing_date,
    "Brand": brand,
    "Article": article,
    "Site": site,
    "Site Area": site_area,
    "MRP": mrp,
    "Promotion Name": promotion_name,
    "Region": region,
    "POS Billing Quantity": pos_billing_quantity,
    "POS Cost": pos_cost,
    "POS Gross Sales": pos_gross_sales,
    "POS Total Discount": pos_total_discount,
    "POS Tax Amount": pos_tax_amount,
    "POS Net Sales WOT": pos_net_sales_wot,
    "POS RGM": pos_rgm,
    "Vendor": vendor,
    "Fiber": fiber,
    "Fabric": fabric,
    "Brand Tag": [f"{b} {f} Saree" for b, f in zip(brand, fabric)],
    "Wash Care Instructions": ["Dry Clean only" if f in ["Silk", "Cotton/Silk"] else "Handwash Separately" for f in fiber],
    "C Technique": craft_code,
    "Broad Craft": ["B1", "B1", "B1", "B2", "B2", "B2", "B3", "B3", "B3", "B4"] * (num_rows // 10),
    "Handloom Mark": random.choices(["yes", "no"], k=num_rows),
    "Silk Mark": ["yes" if f == "Silk" else "no" for f in fiber],
    "Type": type,
    "Style Code": style_code,
    "Craft Code": craft_code,
    "Site Name": site_name,
    "Bin": bin
}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv("sample_dataset.csv", index=False)
df

,Billing Date,Brand,Article,Site,Site Area,MRP,Promotion Name,Region,POS Billing Quantity,POS Cost,...,Wash Care Instructions,C Technique,Broad Craft,Handloom Mark,Silk Mark,Type,Style Code,Craft Code,Site Name,Bin
0,2023-04-05,Biba,4202673296,ST18,B,5675,P24,R4,1,0.460985,...,Handwash Separately,C5,B1,yes,no,Woven,4202673,C5,ST18B,5000-5999
1,2023-03-20,Biba,8164438132,ST01,C,3535,P14,R2,1,0.495580,...,Handwash Separately,C6,B1,yes,no,Printed,8164438,C6,ST01C,3000-3999
2,2023-01-07,Ciba,3740806657,ST29,B,9833,P34,R4,1,0.455052,...,Handwash Separately,C3,B1,yes,no,Embroidered,3740806,C3,ST29B,9000-9999
3,2023-02-05,Biba,9106211375,ST24,A,2513,P16,R2,1,0.462515,...,Handwash Separately,C2,B2,yes,no,Woven,9106211,C2,ST24A,2000-2999
4,2023-01-23,Aiba,5854277108,ST34,B,9770,P23,R2,1,0.488732,...,Handwash Separately,C4,B2,yes,no,Tie-Dyed,5854277,C4,ST34B,9000-9999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-01-04,Aiba,3206215283,ST31,A,6938,P8,R1,1,0.494103,...,Handwash Separately,C5,B2,no,no,Tie-Dyed,3206215,C5,ST31A,6000-6999
996,2023-03-13,Aiba,9887517078,ST02,B,1061,P14,R4,1,0.486100,...,Handwash Separately,C7,B3,yes,no,Woven,9887517,C7,ST02B,1000-1999
997,2023-02-13,Biba,2688104341,ST39,A,5971,P26,R4,0,0.000000,...,Handwash Separately,C7,B3,no,no,Woven,2688104,C7,ST39A,5000-5999
998,2023-01-13,Ciba,7586408854,ST27,B,8852,P2,R4,1,0.452247,...,Dry Clean only,C6,B3,no,yes,Woven,7586408,C6,ST27B,8000-8999
